### Process documents for evaluation (Dont run file unless necessary - will write files to directory)

In [3]:
import sqlite3
import os
from langchain_community.utilities.sql_database import SQLDatabase
from pathlib import Path
import json
from datasets import load_dataset
import pandas as pd



/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Write to .sql file

##### Helper Functions

In [3]:
def sql_to_db(sql_file, new_db_filename):
    conn = sqlite3.connect('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/' + new_db_filename + '.db')

    # Read and execute the schema.sql file
    with open(sql_file, 'r') as f:
        conn.executescript(f.read())

    conn.commit()
    conn.close()

def get_schema(_):
    return DB.get_table_info()


def sql_filepaths(directory_path, output_filepath):
    with open(output_filepath, 'w') as outfile:
        for root, dirs, files in os.walk(directory_path):
            # List subdirectories directly within the current directory
            if root == os.path.join(directory_path):
                for directory in dirs:
                    # print("DIR", directory)

                    directory_files = os.listdir(os.path.join(directory_path, directory))
                    # print("__Files in", directory)
                    for file in directory_files:
                        if file.endswith('.sql'):
                            file_path = os.path.join(root, directory, file)
                            # print(file)
                            outfile.write(file_path + '\n')

In [4]:
# directory_path = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/test_database'

# # Output file to write the paths
# output_file = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/sql_files.txt'


sql_filepaths(directory_path='/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/test_database', output_filepath='/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/sql_files.txt')

##### Create .db

- Read sql_files.txt 
- index into file name
- build .db file with filename

In [5]:
erroneous_files = []

sql_file = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/sql_files.txt'

def build_dbs(sql_txt_file):
    with open(sql_txt_file, 'r') as file:
        # Read each line in the file
        lines = file.readlines()
        
        for line in lines:

            try:

                line = line.strip()
                new_filename = line[84:-11]              #naively index into the name of the database given absolute path
                sql_to_db(line,new_db_filename=new_filename)

            except:
                print("Error occurred: " + new_filename)
                erroneous_files.append(new_filename)


# build_dbs(sql_txt_file=sql_file)

In [6]:
DB = SQLDatabase.from_uri("sqlite:////Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/aan_1.db")

len(get_schema(''))


0

In [7]:
schemas = []
valid_datasets = []
erroneous_datasets = []

In [8]:
s = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/aan_1.db'
s[81:]

'aan_1.db'

In [9]:
directory_path = Path('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/')

schemas = []
valid_datasets = []
erroneous_datasets = []

for file_path in directory_path.iterdir():
    # Check if it's a file (not a directory)
    try:
        DB = SQLDatabase.from_uri("sqlite:////" + str(file_path))

        # Process the file
        # print(f"Processing file: {file_path}")
        x = get_schema('')
        if len(x) > 0:
            schemas.append((str(file_path)[81:],x))
            valid_datasets.append(str(file_path))

            # Add your file processing logic here
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        erroneous_datasets.append(str(file_path))
        # Skip to the next iteration
        continue





/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(


Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/store_product.db: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'


/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(
/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Cust_ID', 'customer', 'Cust_ID')' could not be located in PRAGMA foreign_keys for table loan
  self._metadata.reflect(


Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/cre_Students_Information_Systems.db: fromisoformat: argument must be str
Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/loan_1.db: Could not initialize target column for ForeignKey 'customer.Cust_ID' on table 'loan': table 'customer' has no column named 'Cust_ID'
Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/soccer_1.db: Player
Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/restaurants.db: Could not initialize target column for ForeignKey 'RESTAURANT.RESTAURANT_ID' on table 'LOCATION': table 'RESTAURANT' has no column named 'RESTAURANT_ID'
Error processing file /Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/book_1.db: Could not initialize target column for ForeignKey 'Author.idAuthorA'

/Users/mymac/opt/anaconda3/envs/LLM/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  self._metadata.reflect(


In [10]:
for entry in erroneous_datasets:
    print(entry[81:-3])
# valid_datasets

store_product
cre_Students_Information_Systems
loan_1
soccer_1
restaurants
book_1
soccer_1.db-jour
car_racing
flight_1
music_1


In [11]:
len('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/')

81

#### Write valid and invalid databases to .txt files

In [12]:
with open('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/valid_datasets.txt', 'a') as file:
    # Append content to the file
    for entry in valid_datasets:
        
        file.write(entry[81:-3] + "\n")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/valid_datasets.txt'

In [ ]:
with open('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/invalid_datasets.txt', 'a') as file:
    # Append content to the file
    for entry in erroneous_datasets:
        
        file.write(entry[81:-3] + "\n")

In [ ]:
len('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/spider/db_schemas/')

81

In [ ]:
schemas

[('boat_1.db',
  '\nCREATE TABLE "Boats" (\n\tbid INTEGER, \n\tname TEXT, \n\tcolor TEXT, \n\tPRIMARY KEY (bid)\n)\n\n/*\n3 rows from Boats table:\nbid\tname\tcolor\n\n*/\n\n\nCREATE TABLE "Reserves" (\n\tsid INTEGER, \n\tbid INTEGER, \n\tday TEXT, \n\tFOREIGN KEY(sid) REFERENCES "Sailors" (sid), \n\tFOREIGN KEY(bid) REFERENCES "Boats" (bid)\n)\n\n/*\n3 rows from Reserves table:\nsid\tbid\tday\n\n*/\n\n\nCREATE TABLE "Sailors" (\n\tsid INTEGER, \n\tname TEXT, \n\trating INTEGER, \n\tage INTEGER, \n\tPRIMARY KEY (sid)\n)\n\n/*\n3 rows from Sailors table:\nsid\tname\trating\tage\n\n*/'),
 ('program_share.db',
  '\nCREATE TABLE broadcast (\n\t"Channel_ID" INTEGER, \n\t"Program_ID" INTEGER, \n\t"Time_of_day" TEXT, \n\tPRIMARY KEY ("Channel_ID", "Program_ID"), \n\tFOREIGN KEY("Program_ID") REFERENCES program ("Program_ID"), \n\tFOREIGN KEY("Channel_ID") REFERENCES channel ("Channel_ID")\n)\n\n/*\n3 rows from broadcast table:\nChannel_ID\tProgram_ID\tTime_of_day\n1\t1\tMorning\n2\t1\tNight\n

#### Write to json for easy access to schema

In [ ]:
json_dict = {}
for s in schemas:
    json_dict[s[0]] = s[1]

file_path = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/schemas.json'


with open(file_path, 'w') as json_file:
    json.dump(json_dict, json_file, indent=4) 

#### Next

In [ ]:
with open('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/valid_datasets.txt', 'r') as file:
# Step 2: Read each line and add it to a list
    lines = file.readlines()
lines = [line.strip() for line in lines]


In [ ]:
lines

['boat_1',
 'program_share',
 'candidate_poll',
 'academic',
 'school_finance',
 'phone_1',
 'bike_1',
 'aircraft',
 'allergy_1',
 'architecture',
 'csu_1',
 'train_station',
 'musical',
 'country_language',
 'station_weather',
 'tvshow',
 'department_store',
 'headphone_store',
 'storm_record',
 'pilot_record',
 'flight_company',
 'e_commerce',
 'conference',
 'pets_1',
 'pilot_1',
 'college_3',
 'course_teach',
 'restaurant_bills',
 'district_spokesman',
 'shop_membership',
 'car_road_race',
 'ship_1',
 'club_leader',
 'soccer_3',
 'customers_card_transactions',
 'solvency_ii',
 'party_people',
 'cre_Theme_park',
 'cre_Doc_Template_Mgt',
 'government_shift',
 'manufacturer',
 'browser_web',
 'soccer_2',
 'dog_kennels',
 'address_1',
 'wine_1',
 'roller_coaster',
 'cre_Doc_Tracking_DB',
 'video_game',
 'party_host',
 'driving_school',
 'real_estate_rentals',
 'hr_1',
 'culture_company',
 'match_season',
 'vehicle_driver',
 'customers_and_products_contacts',
 'cre_Doc_Control_Systems',

In [15]:
valid_txt = '/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/helper_txt_files/valid_datasets.txt'

dataset = load_dataset("xlangai/spider")

dataset_train = dataset['train'].to_pandas()    
dataset_validation = dataset['validation'].to_pandas()



In [34]:
concert_singer_Qs = []
for idx, row in dataset_validation.iterrows():
        if row['db_id'] == 'concert_singer':
            concert_singer_Qs.append(row['question'])

# print(list(new_train['question']))


In [37]:
concert_singer_Qs

['How many singers do we have?',
 'What is the total number of singers?',
 'Show name, country, age for all singers ordered by age from the oldest to the youngest.',
 'What are the names, countries, and ages for every singer in descending order of age?',
 'What is the average, minimum, and maximum age of all singers from France?',
 'What is the average, minimum, and maximum age for all French singers?',
 'Show the name and the release year of the song by the youngest singer.',
 'What are the names and release years for all the songs of the youngest singer?',
 'What are all distinct countries where singers above age 20 are from?',
 'What are  the different countries with singers above age 20?',
 'Show all countries and the number of singers in each country.',
 'How many singers are from each country?',
 'List all song names by singers above the average age.',
 'What are all the song names by singers who are older than average?',
 'Show location and name for all stadiums with a capacity 

In [23]:
new_test['db_id'].unique()

array(['tvshow', 'pets_1', 'course_teach', 'cre_Doc_Template_Mgt',
       'dog_kennels', 'orchestra', 'museum_visit', 'car_1'], dtype=object)

In [ ]:
X_train = []
X_test = []

for idx, row in new_train.iterrows():
    entry = (row['db_id'],row['question'])
    X_train.append(entry)

for idx, row in new_test.iterrows():
    entry = (row['db_id'],row['question'])
    X_test.append(entry)

In [ ]:
X_test

[('tvshow', 'List the title of all cartoons in alphabetical order.'),
 ('tvshow', 'What are the titles of the cartoons sorted alphabetically?'),
 ('tvshow', 'List all cartoon directed by "Ben Jones".'),
 ('tvshow', 'What are the names of all cartoons directed by Ben Jones?'),
 ('tvshow', 'How many cartoons were written by "Joseph Kuhr"?'),
 ('tvshow', 'What is the number of cartoones written by Joseph Kuhr?'),
 ('tvshow',
  'list all cartoon titles and their directors ordered by their air date'),
 ('tvshow',
  'What is the name and directors of all the cartoons that are ordered by air date?'),
 ('tvshow',
  'List the title of all cartoon directed by "Ben Jones" or "Brandon Vietti".'),
 ('tvshow',
  'What are the titles of all cartoons directed by Ben Jones or Brandon Vietti?'),
 ('tvshow',
  'Which country has the most of TV Channels? List the country and number of TV Channels it has.'),
 ('tvshow',
  'What is the country with the most number of TV Channels and how many does it have?')

In [ ]:
new_test

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,tvshow,SELECT Title FROM Cartoon ORDER BY title,List the title of all cartoons in alphabetical...,"[SELECT, Title, FROM, Cartoon, ORDER, BY, title]","[select, title, from, cartoon, order, by, title]","[List, the, title, of, all, cartoons, in, alph..."
1,tvshow,SELECT Title FROM Cartoon ORDER BY title,What are the titles of the cartoons sorted alp...,"[SELECT, Title, FROM, Cartoon, ORDER, BY, title]","[select, title, from, cartoon, order, by, title]","[What, are, the, titles, of, the, cartoons, so..."
2,tvshow,SELECT Title FROM Cartoon WHERE Directed_by = ...,"List all cartoon directed by ""Ben Jones"".","[SELECT, Title, FROM, Cartoon, WHERE, Directed...","[select, title, from, cartoon, where, directed...","[List, all, cartoon, directed, by, ``, Ben, Jo..."
3,tvshow,SELECT Title FROM Cartoon WHERE Directed_by = ...,What are the names of all cartoons directed by...,"[SELECT, Title, FROM, Cartoon, WHERE, Directed...","[select, title, from, cartoon, where, directed...","[What, are, the, names, of, all, cartoons, dir..."
4,tvshow,SELECT count(*) FROM Cartoon WHERE Written_by ...,"How many cartoons were written by ""Joseph Kuhr""?","[SELECT, count, (, *, ), FROM, Cartoon, WHERE,...","[select, count, (, *, ), from, cartoon, where,...","[How, many, cartoons, were, written, by, ``, J..."
...,...,...,...,...,...,...
895,car_1,SELECT CountryName FROM countries EXCEPT SELEC...,What are the names of the countries with no ca...,"[SELECT, CountryName, FROM, countries, EXCEPT,...","[select, countryname, from, countries, except,...","[What, are, the, names, of, the, countries, wi..."
896,car_1,"select t1.id , t1.maker from car_makers as t1...",Which are the car makers which produce at leas...,"[select, t1.id, ,, t1.maker, from, car_makers,...","[select, t1, ., id, ,, t1, ., maker, from, car...","[Which, are, the, car, makers, which, produce,..."
897,car_1,"SELECT T1.Id , T1.Maker FROM CAR_MAKERS AS T1...",What are the ids and makers of all car makers ...,"[SELECT, T1.Id, ,, T1.Maker, FROM, CAR_MAKERS,...","[select, t1, ., id, ,, t1, ., maker, from, car...","[What, are, the, ids, and, makers, of, all, ca..."
898,car_1,"SELECT T1.countryId , T1.CountryName FROM Cou...",What are the id and names of the countries whi...,"[SELECT, T1.countryId, ,, T1.CountryName, FROM...","[select, t1, ., countryid, ,, t1, ., countryna...","[What, are, the, id, and, names, of, the, coun..."


In [ ]:
X_test[0]

('tvshow', 'List the title of all cartoons in alphabetical order.')

In [ ]:
for x in X_test:
    print(x[0])

tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
tvshow
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
pets_1
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach
course_teach

In [ ]:
def read_json_file(file_path):
    """
    Read a JSON file and return its content as a dictionary.
    
    Parameters:
        file_path (str): The path to the JSON file.
    
    Returns:
        dict: The content of the JSON file as a dictionary.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [ ]:

SCHEMAS_DICT = read_json_file('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/info_files/schemas.json')


In [ ]:
SCHEMAS_DICT

{'boat_1.db': '\nCREATE TABLE "Boats" (\n\tbid INTEGER, \n\tname TEXT, \n\tcolor TEXT, \n\tPRIMARY KEY (bid)\n)\n\n/*\n3 rows from Boats table:\nbid\tname\tcolor\n\n*/\n\n\nCREATE TABLE "Reserves" (\n\tsid INTEGER, \n\tbid INTEGER, \n\tday TEXT, \n\tFOREIGN KEY(sid) REFERENCES "Sailors" (sid), \n\tFOREIGN KEY(bid) REFERENCES "Boats" (bid)\n)\n\n/*\n3 rows from Reserves table:\nsid\tbid\tday\n\n*/\n\n\nCREATE TABLE "Sailors" (\n\tsid INTEGER, \n\tname TEXT, \n\trating INTEGER, \n\tage INTEGER, \n\tPRIMARY KEY (sid)\n)\n\n/*\n3 rows from Sailors table:\nsid\tname\trating\tage\n\n*/',
 'program_share.db': '\nCREATE TABLE broadcast (\n\t"Channel_ID" INTEGER, \n\t"Program_ID" INTEGER, \n\t"Time_of_day" TEXT, \n\tPRIMARY KEY ("Channel_ID", "Program_ID"), \n\tFOREIGN KEY("Program_ID") REFERENCES program ("Program_ID"), \n\tFOREIGN KEY("Channel_ID") REFERENCES channel ("Channel_ID")\n)\n\n/*\n3 rows from broadcast table:\nChannel_ID\tProgram_ID\tTime_of_day\n1\t1\tMorning\n2\t1\tNight\n3\t2\tM

In [2]:
import re

# Read the content of the text file
with open('/Users/mymac/LLM/Personal-Medical-Assistant/backend/evaluation/executable/spider_eval/eval_txts/mistralai/Mistral-7B-Instruct-v0.1_pred_200.txt', 'r') as file:
    content = file.read()



content = content.replace('\n', '')

# Split by ;
queries = content.split(';')

# Add a newline character at the end of each entry
queries = [query.strip() + ';\n' for query in queries if query.strip()]

# Write the reformatted content back to the text file
with open('reformatted_file.txt', 'w') as file:
    file.writelines(queries)

In [5]:
def sql_to_db(sql_file, new_db_filename):
    conn = sqlite3.connect('/Users/mymac/LLM/Personal-Medical-Assistant/database' + new_db_filename + '.db')

    # Read and execute the schema.sql file
    with open(sql_file, 'r') as f:f
        conn.executescript(f.read())

    conn.commit()
    conn.close()


In [6]:
sql_to_db('/Users/mymac/LLM/Personal-Medical-Assistant/database/SQL_medical_assistant.sql', 'SQL_medical_assistant')

OperationalError: near "SCHEMA": syntax error